# 各种杂项代码
TODO: 将代码规范化后放入工具模块中

In [6]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime as dt
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import time

# 回测框架
import backtrader as bt

# 基础函数
import utilsJ

## 基本面

### 因子

#### 盈利性
+ 毛利率 - netprofit_margin
+ 净利率 - grossprofit_margin
+ 扣非净资产收益率 - roe_dt
+ 营收同比增长率 - or_yoy
+ 净利润同比增长率 - dt_netprofit_yoy (归母扣非净利润同比增长率)
+ 扣非净利润 - dtprofit_to_profit

#### 流动性
+ 资产负债率 - debt_to_assets
+ 流动比率 - current_ratio
+ 速动比率 - quick_ratio
+ 应收账款周转率 - ar_turn
+ 应收账款账龄分布
+ 存货周转率 - inv_turn

#### 估值
+ PE倍数（市盈率）- pe
+ PB倍数（市净率）- pb
+ PEG

#### 其他
+ 商誉占总资产比重 goodwill / total_assets
+ 商誉占净资产比重 goodwill / (total_assets - total_liab)

### 数据获取

In [ ]:
def fund_ind(stock_code,
             start=dt.datetime.now()-dt.timedelta(days=365),
             end=dt.datetime.now()):
    pro = ts.pro_api(utilsJ.load_token())

    ### Finance Indicators
    find_fields = ['netprofit_margin', 'grossprofit_margin', 'roe_dt', 'or_yoy', 'dt_netprofit_yoy', 'dtprofit_to_profit',
                   'debt_to_assets', 'current_ratio', 'quick_ratio', 'ar_turn', 'inv_turn']
    df_find = pro.fina_indicator(ts_code=stock_code, 
                                 start_date=start.strftime('%Y%m%d'),
                                 end_date=end.strftime('%Y%m%d'))
    non_index_list = [x for x in find_fields if x not in df_find.columns]
    df_find = df_find[['end_date'] + [x for x in find_fields if x in df_find.columns]].drop_duplicates()
    for non_index in non_index_list:
        df_find[non_index] = 0
    df_find.rename(columns = {'end_date':'trade_date'}, inplace = True)
    

    ### Balance Sheets
    df_bs = pro.balancesheet(ts_code=stock_code, 
                         start_date=start.strftime('%Y%m%d'),
                         end_date=end.strftime('%Y%m%d'), 
                         fields='end_date, goodwill, total_assets, total_liab')
    df_bs['goodwill'] = df_bs['goodwill'].fillna(0)
    df_bs['gw/a'] = df_bs.goodwill / df_bs.total_assets
    df_bs['gw/n'] = df_bs.goodwill / (df_bs.total_assets - df_bs.total_liab)
    df_bs.drop(columns=['goodwill', 'total_assets', 'total_liab'], inplace=True)
    df_bs.rename(columns={'end_date':'trade_date'}, inplace=True)

    ### Daliy
    df_daily = pro.daily_basic(ts_code=stock_code, 
                           start_date=start.strftime('%Y%m%d'),
                           end_date=end.strftime('%Y%m%d'), 
                           fields='trade_date, pe, pb')[::-1]
    df_daily['pe'] = df_daily['pe'].fillna(0)

    df_final = df_daily.merge(df_find, on='trade_date', how='left').merge(df_bs, on='trade_date', how='left')
    df_final['trade_date'] = pd.to_datetime(df_final.trade_date)
    df_final.index=pd.to_datetime(df_final.trade_date)
    df_final.drop('trade_date', axis=1, inplace = True)
    df_final.fillna(method='bfill', inplace=True)
    df_final.fillna(method='pad', inplace=True)
    return df_final

### 全市场测试

In [ ]:
condition = {
    'netprofit_margin' : (1, 30),
    'grossprofit_margin' : (1, 20),
    'roe_dt' : (1, 15),
    'or_yoy' : (1, 15),
    'dt_netprofit_yoy' : (1, 15),
    'debt_to_assets' : (-1, 50),
    'current_ratio' : (1, 2),
    'quick_ratio' : (1, 2),
    'pe' : (1, 30),
    'pb' : (1, 1)
}
startdate = dt.datetime.now() - dt.timedelta(days=365)
enddate = dt.datetime.now()

stock_list = utilsJ.get_stock_list()

comp_per = {}
for s_code in stock_list:
    try:
        comp_per[s_code] = 0
        df = fund_ind(s_code, startdate, enddate)
        if len(df) != 0:
            for col in df.columns:
                if col in condition.keys() and df.loc[:, col][-1] != 0:
                    if condition[col][0] == 1:
                        comp_per[s_code] += 1 if df.loc[:, col][-1] >= condition[col][1] else 0
                    else:
                        comp_per[s_code] += 1 if df.loc[:, col][-1] <= condition[col][1] else 0
            print((s_code, comp_per[s_code]))
            time.sleep(4)
    except:
        print(s_code)

sorted(comp_per.items(), key=lambda x:x[1], reverse=True)

## 爬虫

抓取东方财富网上的资金流向

In [4]:
def money_flow(stock_code):
    sec_id = '0.' + stock_code[:6] if stock_code[0] != '6' else '1.' + stock_code[:6]
    headers = {'Accept': '*/*',
            'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
            'Host': 'push2his.eastmoney.com',
            'Referer': 'https://data.eastmoney.com/',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.47',
                }

    f_dict = {'f51': 'datetime',
            'f52': 'major_net_amount',
            'f53': 'sm_net_amount',
            'f54': 'md_net_amount',
            'f55': 'lg_net_amount',
            'f56': 'elg_net_amount',
            'f57': 'major_net_percent',
            'f58': 'sm_net_percent',
            'f59': 'md_net_percent',
            'f60': 'lg_net_percent',
            'f61': 'elg_net_percent'
        }

    params = (
            ('lmt', '0'),
            ('klt', '101'),
            ('secid','0.002514'),
            ('fields1', 'f1,f2,f3,f7'),
            ('fields2', ",".join(f_dict.keys())),
        )
    params = dict(params)
    url = 'http://push2his.eastmoney.com/api/qt/stock/fflow/daykline/get'
    json_response = requests.get(url, headers=headers, params=params).json()
    data = json_response.get('data')
    df = pd.DataFrame([x.split(',') for x in data['klines']], columns=f_dict.values())
    df['datetime'] = df.datetime.apply(lambda x: dt.datetime.strptime(x, '%Y-%m-%d'))
    return  df

In [7]:
money_flow('002514.SZ')

,datetime,major_net_amount,sm_net_amount,md_net_amount,lg_net_amount,elg_net_amount,major_net_percent,sm_net_percent,md_net_percent,lg_net_percent,elg_net_percent
0,2022-06-27,17874368.0,-9255281.0,-8619086.0,-8867172.0,26741540.0,35.95,-18.61,-17.33,-17.83,53.78
1,2022-06-28,-12523739.0,30063426.0,-17539692.0,-42957571.0,30433832.0,-1.80,4.32,-2.52,-6.18,4.38
2,2022-06-29,-260446084.0,194984208.0,65461872.0,-56347568.0,-204098516.0,-13.90,10.41,3.49,-3.01,-10.89
3,2022-06-30,51778038.0,7168768.0,-58946800.0,-100770864.0,152548902.0,2.88,0.40,-3.28,-5.61,8.49
4,2022-07-01,44234144.0,35023632.0,-79257792.0,-102626784.0,146860928.0,1.98,1.57,-3.55,-4.59,6.57
...,...,...,...,...,...,...,...,...,...,...,...
96,2022-11-16,-222847716.0,201401628.0,21446080.0,-72092838.0,-150754878.0,-22.01,19.89,2.12,-7.12,-14.89
97,2022-11-17,-22026813.0,34041617.0,-12014804.0,-1539695.0,-20487118.0,-4.30,6.64,-2.34,-0.30,-4.00
98,2022-11-18,84463163.0,-42358560.0,-42104608.0,5591337.0,78871826.0,7.70,-3.86,-3.84,0.51,7.19
99,2022-11-21,95367534.0,-44327024.0,-51040528.0,-59483360.0,154850894.0,6.13,-2.85,-3.28,-3.83,9.96
